# Tank Sizing Python Code for Propellants and Pressurant Using Assumptions
### Note: Only N2 for pressurant, LOX for oxidizer, and kerosene for fuel (SO FAR)
### Note: Only isothermal assumption coded (SO FAR)

In [168]:
import math
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# LOX Tank Volume Calculator
### Note: Density as a function of temperature and pressure yet to be implemented
Density taken at boiling point of O2

In [169]:
burntime=10 # seconds
massflow_pressurant=1.7779 #kg/s
OF=2.2 # Oxidizer to Fuel ratio
ullageRatio=1.15 # percentage of tank that is ullage
#T= # in Kelvin
density=1097.26 #kg/m^3

def loxVolume(burntime, massflow_pressurant, OF, ullageRatio):
    massflow=massflow_pressurant*OF/(OF+1)
    mass=burntime*massflow
    volume=mass/density
    volumeFinal=volume*ullageRatio
    print("lox massflow = " + str(massflow) + " kg/s")
    print("lox massflow = " + str(massflow*2.20462) + " lb/s")
    print("lox mass = " + str(mass) + " kg")
    print("lox mass = " + str(mass*2.20462) + " lb")
    print("lox tank volume = " + str(volumeFinal) + " m^3")
    print("lox tank volume = " + str(volumeFinal*35.3147) + " ft^3")
    

loxVolume(burntime,massflow_pressurant,OF,ullageRatio)

lox massflow = 1.22230625 kg/s
lox massflow = 2.6947208048749998 lb/s
lox mass = 12.2230625 kg
lox mass = 26.947208048749996 lb
lox tank volume = 0.012810566205821772 m^3
lox tank volume = 0.45240130238873416 ft^3


# Kerosene Tank Volume Calculator
### Note: Density as a function of temperature and pressure yet to be implemented
Density taken at room temp. (298 K)

In [170]:
burntime=10 # seconds
massflow_pressurant=1.77 #kg/s
OF=2.2 # Oxidizer to Fuel ratio
ullageRatio=1.15 # percentage of tank that is ullage
#T= # in Kelvin
density=820 #kg/m^3

def keroVolume(burntime, massflow_pressurant, OF, ullageRatio):
    massflow=massflow_pressurant*1/(OF+1)
    mass=burntime*massflow
    volume=mass/density
    volumeFinal=volume*ullageRatio
    print("kerosene massflow = " + str(massflow) + " kg/s")
    print("kerosene massflow = " + str(massflow*2.20462) + " kg/s")
    print("kerosene mass = " + str(mass) + " kg")
    print("kerosene mass = " + str(mass*2.20462) + " lb")
    print("kerosene tank volume = " + str(volumeFinal) + " m^3")
    print("kerosene tank volume = " + str(volumeFinal*35.3147) + " ft^3")
    
keroVolume(burntime,massflow_pressurant,OF,ullageRatio)

kerosene massflow = 0.553125 kg/s
kerosene massflow = 1.2194304374999998 kg/s
kerosene mass = 5.53125 kg
kerosene mass = 12.194304375 lb
kerosene tank volume = 0.007757240853658536 m^3
kerosene tank volume = 0.2739446335746951 ft^3


# Tank Height Needed to House Propellant (With Ellipsoidal Ends)
### Note: Does not account for thickness, thickness of tank assumed to be 0 in

In [171]:
semi_axis=1 #in
radius=3 #in
propVolume=0.2739 #ft^3 (ullage ratio factored in from keroVolume function)

def elpsTankHeight(semi_axis,radius, propVolume):
    propVolume=propVolume*12**3 #ft^3 to in^3
    print("propellant tank volume " + str(propVolume) + " in^3")
    elpsCapVolume=2*2/3*math.pi*radius**2*semi_axis #volume of both caps combined
    print("volume of ellipsoidal caps = " + str(elpsCapVolume) + " in^3")
    elpsTankHeight=(propVolume - elpsCapVolume)/math.pi/radius**2 #solve for tank height in between caps
    print("cylinder tank height for 6 in diameter with " + str(semi_axis) + " in ellipsoidal caps is: " + str(elpsTankHeight) + " in")

elpsTankHeight(semi_axis, radius, propVolume)

propellant tank volume 473.2992 in^3
volume of ellipsoidal caps = 37.69911184307752 in^3
cylinder tank height for 6 in diameter with 1 in ellipsoidal caps is: 15.406201609208798 in


# Pressurant Tank Volume Functions

### Note: Compressibility and Volume Functions only run with at least two pressure values inputted, if you only want to try a single pressure use P=np.ones(2)s*(pressure)
### Note: Pressure must input in psi, Temperature must be input in Kelvin, mass must be in kg

## Functions:

In [172]:
def calculate_compressibility_factor(temperature, pressure):
    # Constants for nitrogen (N2)
    critical_temperature = 126.2  # K
    critical_pressure = 3.39  # MPa
    pressure=pressure*6894.76/1e6 # convert from psi

    # Redlich-Kwong parameters
    a = 0.42748 * (critical_temperature ** 2.5) / critical_pressure
    b = 0.08664 * critical_temperature / critical_pressure

    # Calculate A and B terms in the Redlich-Kwong equation
    A = a * pressure / (8.314 * temperature ** 2)
    B = b * pressure / (8.314 * temperature)

    n=len(pressure)
    Z=np.zeros(n)

    # Solve the cubic equation to find the compressibility factor
    for i in range(0,len(pressure)):
        coefficients = [1, -1, A[i] - B[i] - B[i]**2, -A[i] * B[i]]
        roots = np.roots(coefficients)
        
        # Filter the real and positive roots (physical solutions)
        valid_roots = [root for root in roots if root.real > 0 and root.imag == 0]

        if not valid_roots:
            return None
        else:
            Z[i] = max(valid_roots)
        
    return Z

def calculate_nitrogen_volume(mass, T, P, Z):
    molar_mass_nitrogen = 0.0280134  # kg/mol
    R = 8.314  # J/(mol·K)

    P=P*6894.76 # convert from psi to Pa

    moles_of_nitrogen = mass / molar_mass_nitrogen

    volume = (Z * moles_of_nitrogen * R * T) / P

    volume = volume * 35.3147 # convert from m^3 to ft^3

    return volume


# Isothermal Assumption
$$ \frac{P_i V_i}{m_i R T_i} = \frac{P_f V_f}{m_f R T_f} $$
$$ T_f=T_i $$
$$ V_f=V_i $$

Control Volume is N2 tank only

$$ C_{backflow} = 1.2 $$

Not based on any documentation, just a safety factor

$$ P_f = C_{backflow} P_{propellant} = (1.2)(600psi) = 720 psi$$
$$ m_f = m_i - \dot{m}_{pressurant} t_b $$
$$ \frac{P_i}{m_i} = \frac{720 psi}{m_i - (1.7779 \frac{kg}{s})(10 s)} $$

We want to find pressure and mass in order to calculate volume. A new constraint is necessary to solve system of equations. Possible constraints: \
Max Tank dimensions (Volume constraint) \
Max Tank thickness (Pressure constraint) \
Max Amount of N2 we want to use (Mass constraint)

Without any constraints, we can solve by arbitrarily choosing either an initial mass or initial pressure for the tank.

To find roots, rearrange equation:

$$ 

In [173]:
from scipy.optimize import fsolve

massflow_pressurant = 1.7779                    # kg/s
burntime = 10                                   # s
psi2pa = 6894.76                                # convert psi to Pa
initial_pressure = 6000                         # psi
final_pressure = 720                            # psi


def calculate_nitrogen_mass_isot(massflow_pressurant, burntime, initial_pressure, final_pressure):

    def equation(x):
        return initial_pressure*psi2pa/x - final_pressure*psi2pa/(x - massflow_pressurant*burntime)

    initial_guess = 6                           # kg

    mass = fsolve(equation,initial_guess)
    print('residual = {}'.format(equation(mass)))
    print('mass of N2 = {}'.format(mass))

    return mass

mass=calculate_nitrogen_mass_isot(massflow_pressurant, burntime, initial_pressure, final_pressure)


residual = [-2.32830644e-10]
mass of N2 = [20.20340909]


In [174]:
T = 298
P = np.ones(2)*initial_pressure
Z = calculate_compressibility_factor(T,P)
N2volume = calculate_nitrogen_volume(mass,T,P,Z)

cbcFt2liters = 28.3168

print('N2 tank volume = {} ft^3'.format(N2volume[0]))
print('{} in^3'.format(N2volume[0]*12**3))
print('{} L'.format(N2volume[0]*cbcFt2liters))

N2 tank volume = 0.09016169108313321 ft^3
155.7994021916542 in^3
2.5530905740628667 L


# IN PROGRESS: Isenthalpic Assumption